In [13]:
!pip install -q duckrun --upgrade

In [14]:
import duckrun
con = duckrun.connect('bigdata/tpch.lakehouse/dbo')
onelake_logs_path = 'abfss://bigdata@onelake.dfs.fabric.microsoft.com/tpch.Lakehouse/Files/DiagnosticLogs/OneLake/Workspaces/*/y=*/m=*/d=*/h=*/m=*/*.json'

✅ Using existing Azure Storage token
✅ Using existing Azure Storage token


In [15]:
try:
    con.sql(f"""  
        CREATE VIEW if not exists logs(file) AS SELECT 'dummy';
    SET VARIABLE list_of_files =
    (
    WITH xxxx AS (
        SELECT
        file
        FROM glob('{onelake_logs_path}')
        WHERE file NOT IN (SELECT DISTINCT file FROM logs) ORDER BY file
        
    )
    SELECT list(file) FROM xxxx
    );
        select *exclude(data),data.*, filename as file  from read_json_auto(GETVARIABLE('list_of_files'),hive_partitioning = true,union_by_name = 1,FILENAME = 1)
                """).write.mode("append").option("mergeSchema", "true").saveAsTable('logs')
except Exception as e:
    print(f"An error occurred: {e}")

✅ Using existing Azure Storage token
Writing to Delta table: dbo.logs (mode=append) (engine=rust, schema_mode=merge)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Data appended to dbo.logs


In [21]:
con.sql(""" select originatingApp, count(*)
 from logs
  group by all
  order by 2 desc
   """).show(max_width=5)

┌───────────────────────────────────────────────────────────────┬──────────────┐
│                        originatingApp                         │ count_star() │
│                            varchar                            │    int64     │
├───────────────────────────────────────────────────────────────┼──────────────┤
│ Model                                                         │        11306 │
│ Spark                                                         │         5609 │
│ SqlAnalyticsEndpoint                                          │         5425 │
│ Fabric                                                        │         1570 │
│ object_store/0.12.4                                           │          930 │
│ FabricPortal                                                  │          242 │
│ azsdk-cpp-storage-files-datalake/12.12.0 (Windows 10 Enterp…  │          194 │
│ RuntimeLineage                                                │          132 │
│ azsdk-cpp-storage-blobs/12

In [17]:
con.sql(""" select 
get_workspace_name(workspaceid) as workspace_name,
get_lakehouse_name(workspaceid, itemId) as lakehouse_name,
originatingApp,operationName,httpStatusCode , 
regexp_extract(resource, '([^&/]+)/([^&/]+)/(Tables|Files)(?:/([^&/]+))?(?:/([^&/]+))?', 4) AS schema_name, count(*)
 from logs
  group by all
  order by 1,2,3 """).show(max_width=120)

✅ Using cached Fabric API token
✅ Using cached Fabric API token
✅ Using cached Fabric API token
✅ Using cached Fabric API token
✅ Using cached Fabric API token
✅ Using cached Fabric API token


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────────┬────────────────┬──────────────────────┬───┬────────────────┬─────────────┬──────────────┐
│ workspace_name │ lakehouse_name │    originatingApp    │ … │ httpStatusCode │ schema_name │ count_star() │
│    varchar     │    varchar     │       varchar        │   │     int64      │   varchar   │    int64     │
├────────────────┼────────────────┼──────────────────────┼───┼────────────────┼─────────────┼──────────────┤
│ bigdata        │ tpch           │ Fabric               │ … │            206 │ dbo         │          258 │
│ bigdata        │ tpch           │ Fabric               │ … │            200 │             │           19 │
│ bigdata        │ tpch           │ Fabric               │ … │            200 │ dbo         │          158 │
│ bigdata        │ tpch           │ Fabric               │ … │            404 │             │            1 │
│ bigdata        │ tpch           │ Fabric               │ … │            404 │ dbo         │           29 │
│ bigdata        │ 

In [18]:
con.sql(""" select 
regexp_extract(resource, '([^&/]+)/([^&/]+)/(Tables|Files)(?:/([^&/]+))?(?:/([^&/]+))?', 4) AS schema_name,
get_workspace_name(workspaceid) as workspace_name,
get_lakehouse_name(workspaceid, itemId) as lakehouse_name,
originatingApp ,
 count(*) as API_calls
 from logs
  group by all
  order by API_calls desc
  limit 5
   """).show(max_width=140)

┌────────────────┬────────────────┬────────────────┬──────────────────────┬───────────┐
│  schema_name   │ workspace_name │ lakehouse_name │    originatingApp    │ API_calls │
│    varchar     │    varchar     │    varchar     │       varchar        │   int64   │
├────────────────┼────────────────┼────────────────┼──────────────────────┼───────────┤
│ aemo           │ tmp            │ data           │ Model                │     11300 │
│ DiagnosticLogs │ bigdata        │ tpch           │ Spark                │      2605 │
│ CH10000        │ bigdata        │ tpch           │ SqlAnalyticsEndpoint │      2419 │
│ Reports        │ tmp            │ data           │ Spark                │      1131 │
│ aemo           │ tmp            │ data           │ Spark                │       980 │
└────────────────┴────────────────┴────────────────┴──────────────────────┴───────────┘

